1. https://python-forum.io/Thread-Type-hinting-return-type-based-on-parameter

In [65]:
import inspect
from functools import wraps
from typing import Any

def set_type_as_var(variable_name):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs_inner):
            return f(*args, **kwargs_inner)
        
        desired_type = Any

        signature = inspect.getfullargspec(f)
        if signature.kwonlydefaults and variable_name in signature.kwonlydefaults:
            desired_type = type(signature.kwonlydefaults.get(variable_name))
            # probably, you can take into account signature.annotations ... 
            
        if signature.args and variable_name in signature.args:
            if signature.defaults is not None:
                desired_type = type(signature.defaults[signature.args.index(variable_name)])
            else:
                desired_type = signature.annotations.get(variable_name, Any)
            
        # maybe you will need other conditions .... 
        
        wrapper.__annotations__['return'] = desired_type
        return wrapper
    return decorator

In [66]:
@set_type_as_var('x')
def a(x:float):
    pass

